# Bayes Logistic Regression

I train the Bayes logistic model using following methods. 
1. Laplace approximation
2. Assumed density filtering

## 0. Import

In [1]:
using Random
using Statistics
using Distributions
using LinearAlgebra
using Plots
pyplot()

Plots.PyPlotBackend()

## 1. functions

In [2]:
#sigmoid function
σ(ξ) = 1/(1+exp(-ξ))

#logistic model
function logistic_model(w,x)
    wtx = dot(w,x)
    q = σ(wtx)
    y = rand(Binomial(1,q))
    return y
end

#model
p(y,ϕx,w) = y==1 ? σ(dot(w,ϕx)) : 1-σ(dot(w,ϕx))

#gradient of the log likelihood
function ∇wloglik(w,Y,ϕX)
    N = length(Y)
    tmp = zeros(N)
    for n in 1:N
        tmp[n] = Y[n]-σ(dot(w, ϕX[:,n]))
    end
    return ϕX * tmp
end

#gradient descent
function gradient_descent(w_init, max_iter, ϵ, α, ∇obj)
    w = w_init
    p = -∇obj(w)
    for k in 1:max_iter
        p = -∇obj(w)
        if norm(p)<ϵ
            return w
            break
        end
        w = w + α*p
    end
    return w
end

#split the data
function split(X, y, train_size)
    N = length(y)
    N_train = Int(floor(train_size * N))
    N_test = N - N_train
    idx = shuffle(1:N)
    idx_train = idx[1:N_train]
    idx_test = idx[N_train+1:end]
    return X[:,idx_train], y[idx_train], X[:,idx_test], y[idx_test], N_train, N_test
end

#posterior sample (isotropic normal and anistropic mormal distribution)
wpost_samps_iso(w_mean, w_var, n_samps) = rand(MvNormal(w_mean, w_var), n_samps)
wpost_samps_aniso(w_mean, w_prec, n_samps) = rand(MvNormalCanon(w_mean, w_prec), n_samps)   

#predictive didtribution
function ysamps(ϕx, wsamps)
    _, n_samps = size(wsamps)
    preds = zeros(n_samps)
    for i in 1:n_samps
        preds[i] = p(1, ϕx, wsamps[:,i])
    end
    prob = mean(preds)
    return prob>0.5 ? 1 : 0
end

#predict Y for test data
function pred(ϕX, w_mean, w_var, samp_func)
    d,N = size(ϕX)
    n_samps = 5000
    wsamps = samp_func(w_mean, w_var, n_samps)
    Y_preds = zeros(N)
    for n in 1:N
        Y_preds[n] = ysamps(ϕX[:,n], wsamps)
    end
    return Y_preds
end

"""
    1. Laplace Approximation: get_params returns the posterior mean and precision matrix of Gaussian
"""
function get_params(ϕX, Y)
    d,N = size(ϕX)
    
    #calculate the mean
    w_init = zeros(d)
    max_iter = 10000
    ϵ = 1e-1
    α = 0.02
    ∇obj(w) = w - ∇wloglik(w,Y,ϕX) #-loglik
    w_MAP = gradient_descent(w_init, max_iter, ϵ, α, ∇obj)
    
    #calculate the covariance matrix
    prec = I(d)
    ϕx = zeros(d)
    for n in 1:N
        ϕx = ϕX[:,n]
        σwtϕx = σ(dot(w_MAP, ϕx))
        prec += σwtϕx * (1-σwtϕx) * ϕx * ϕx'
    end
    return w_MAP, Matrix(Hermitian(prec))
end

"""
    2. Assumed Density Filtering (ADF): myADF returns the posterior mean and variation of Gaussian
"""
#derivative of logZ
function ∇logZ(ϕx, y, μt, st_sq)
    c = 1+π*st_sq*norm(ϕx)^2/8
    tmp = dot(μt, ϕx)/sqrt(c)
    ∇μtmp = ϕx/sqrt(c)
    ∂stsqtmp = -dot(μt,ϕx)*norm(ϕx)^2/16/c^(3/2)
    ∂μlogZ = y*(1-σ(tmp))*∇μtmp - (1-y)*σ(tmp)*∇μtmp
    ∂ssqlogZ = y*(1-σ(tmp))*∂stsqtmp - (1-y)*σ(tmp)*∂stsqtmp
    return ∂μlogZ, ∂ssqlogZ
end

#update the parameters
function update_params(ϕX, Y, t, μt, st_sq, d)
    ϕx = ϕX[:,t+1]
    y = Y[t+1]
    ∇μlogZ, ∂ssqlogZ = ∇logZ(ϕx, y, μt, st_sq)
    μ_new = μt + st_sq * ∇μlogZ
    s_new = st_sq - st_sq^2 * (norm(∇μlogZ)^2 - 2 * ∂ssqlogZ)/d
    return μ_new, s_new
end

#Assumed density filtering
function myADF(ϕX, Y, w_mean_init, w_var_init)
    d,N = size(ϕX)
    w_means = zeros(d,N)
    w_vars = zeros(N)
    w_means[:,1] = w_mean_init
    w_vars[1] = w_var_init
    
    μt = w_mean_init
    st_sq = w_var_init
    for t in 1:N-1
        μt, st_sq = update_params(ϕX, Y, t, μt, st_sq, d)
        w_means[:,t+1] = μt
        w_vars[t+1] = st_sq
    end
    return w_means[:,end], w_vars[end]
end

myADF (generic function with 1 method)

## 2. create the data

In [3]:
#create the data
Random.seed!(42)
N = 200

#true weights
w_true = [2.1, -0.7, -1.4, 3.2] #true weights
d = length(w_true)

#feature vector
ϕX = zeros(d, N)
ϕX[1,:] = ones(N)
ϕX[2,:] = 3*(rand(N).-0.5)
ϕX[3,:] = 2*rand(Normal(0,2), N)
ϕX[4,:] = rand(Gamma(3,1.5), N) .- 5

#output
Y = zeros(N)
for n in 1:N
    Y[n] = logistic_model(w_true, ϕX[:,n])
end

#split the data
train_size = 0.8
ϕX_train, Y_train, ϕX_test, Y_test, N_train, N_test = split(ϕX, Y, train_size)

#information about the data
println("true weights: $(w_true)")
println("N_train=$(N_train)")
println("N_test=$(N_test)")
println("total => class 1: $(Int(sum(Y))), class 0: $(Int(N-sum(Y)))")
println("train => class 1: $(Int(sum(Y_train))), class 0: $(Int(N_train-sum(Y_train)))")
println("test  => class 1: $(Int(sum(Y_test))), class 0: $(Int(N_test-sum(Y_test)))")

true weights: [2.1, -0.7, -1.4, 3.2]
N_train=160
N_test=40
total => class 1: 100, class 0: 100
train => class 1: 81, class 0: 79
test  => class 1: 19, class 0: 21


## 3. Laplace Approximation

In [4]:
#Laplace Approximation
w_mean, w_prec = get_params(ϕX_train, Y_train)
println("posterior mean of weights: $(round.(w_mean, digits=3))")

posterior mean of weights: [1.383, -0.491, -1.053, 2.445]


In [5]:
#prediction for train set 
Y_train_preds = pred(ϕX_train, w_mean, w_prec, wpost_samps_aniso)
println("train accuracy = $(sum(Y_train .== Y_train_preds)/N_train)")

#prediction for test set
Y_test_preds = pred(ϕX_test, w_mean, w_prec, wpost_samps_aniso)
println("test accuracy  = $(sum(Y_test .== Y_test_preds)/N_train)")

train accuracy = 0.975
test accuracy  = 0.24375


## 4. Assumed Density Filtering

In [6]:
#ADF
w_mean_init = zeros(d)
w_var_init = 1.0
w_mean, w_var = myADF(ϕX_train, Y_train, w_mean_init, w_var_init)
println("posterior mean of weights: $(round.(w_mean, digits=3)).")
println("posterior std of weights: $(round(sqrt(w_var), digits=3))")

posterior mean of weights: [0.813, -0.482, -1.181, 2.509].
posterior std of weights: 0.222


In [7]:
#prediction for train set 
Y_train_preds = pred(ϕX_train, w_mean, w_var, wpost_samps_iso)
println("train accuracy = $(sum(Y_train .== Y_train_preds)/N_train)")

#prediction for test set
Y_test_preds = pred(ϕX_test, w_mean, w_var, wpost_samps_iso)
println("test accuracy  = $(sum(Y_test .== Y_test_preds)/N_train)")

train accuracy = 0.95625
test accuracy  = 0.2375
